In [1]:
import logging
import os
import sys
import random
import json
from datetime import datetime
module_path = os.path.abspath('/home/alex/data/LightClip/CLIP-KD')
if module_path not in sys.path:
    sys.path.append(module_path)

import numpy as np
import torch
from src.open_clip import create_model,AppleMobileCLIP
from src.open_clip.transform import image_transform
from src.open_clip import get_tokenizer

import mobileclip
from mobileclip.models.mci import ParallelAttentionBlock
from PIL import  Image

device = torch.device("cuda")
model = create_model(model_name="timm-swin_tiny_patch4_window7_224",device=torch.device("cuda"))

tokenizer_openclip = get_tokenizer("timm-swin_tiny_patch4_window7_224")




model_name:swin_tiny_patch4_window7_224


/home/alex/.conda/envs/clipenv/lib/python3.10/site-packages/torch/functional.py:504: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at ../aten/src/ATen/native/TensorShape.cpp:3190.)
  return _VF.meshgrid(tensors, **kwargs)  # type: ignore[attr-defined]


In [2]:
mobile_model, _, preprocess = mobileclip.create_model_and_transforms('mobileclip_s0', pretrained='/home/alex/data/LightClip/ml-mobileclip/checkpoints/mobileclip_s0.pt')
preprocess_val = image_transform(256,is_train=False, mean=[0.48145466,0.4578275,0.40821073], std=[0.26862954,0.26130258,0.27577711])


In [3]:
print(preprocess)
print(preprocess_val)

Compose(
    Resize(size=256, interpolation=bilinear, max_size=None, antialias=None)
    CenterCrop(size=(256, 256))
    ToTensor()
)
Compose(
    Resize(size=256, interpolation=bicubic, max_size=None, antialias=None)
    CenterCrop(size=(256, 256))
    <function _convert_to_rgb at 0x75d941f6ad40>
    ToTensor()
    Normalize(mean=[0.48145466, 0.4578275, 0.40821073], std=[0.26862954, 0.26130258, 0.27577711])
)


In [5]:
import torchvision.transforms as transforms
preprocess_val = transforms.Compose([
    transforms.Resize(size=256, interpolation=transforms.InterpolationMode.BICUBIC),
    transforms.CenterCrop(size=(256, 256)),
    transforms.Lambda(lambda image: image.convert('RGB')),  # Assuming _convert_to_rgb is this
    transforms.ToTensor()
])

In [4]:
mobile_model = mobile_model.to(device)
image = preprocess(Image.open("/home/alex/data/LightClip/ml-mobileclip/docs/test_cap.png").convert('RGB')).unsqueeze(0).half().to(device)
tokenizer = mobileclip.get_tokenizer('mobileclip_s0')

text = tokenizer(["a photo of diagram", "a photo of dog", "a photo of cat", "a photo of girl"]).to(device)

with torch.no_grad(), torch.cuda.amp.autocast():
    image_features = mobile_model.encode_image(image)
    text_features = mobile_model.encode_text(text)
    image_features /= image_features.norm(dim=-1, keepdim=True)
    text_features /= text_features.norm(dim=-1, keepdim=True)

    text_probs = (100.0 * image_features @ text_features.T).softmax(dim=-1)

print("Label probs:", text_probs)
for prob in text_probs[0]:
    print(prob.item())

Label probs: tensor([[4.3204e-06, 1.4085e-04, 4.5825e-04, 9.9940e-01]], device='cuda:0')
4.320384960010415e-06
0.00014085338625591248
0.00045824635890312493
0.999396562576294


In [5]:
print(image_features.shape)

torch.Size([1, 512])


In [7]:
image_features @ text_features.t()

tensor([[0.1212, 0.1565, 0.1683, 0.2461]], device='cuda:0',
       dtype=torch.float16)

In [8]:

model = AppleMobileCLIP(**(model.init_params)).to(device)
del model.visual
model.visual = mobile_model.image_encoder.to(device)
model.visual.model.network[7][0] = ParallelAttentionBlock(**model.visual.model.network[7][0].init_params).to(device)
model.visual.model.network[7][1] = model.visual.model.network[7][0]
del model.transformer
model.transformer = mobile_model.text_encoder.to(device)
preprocess_train = image_transform(256,is_train=True, mean=[0.48145466,0.4578275,0.40821073], std=[0.26862954,0.26130258,0.27577711])
preprocess_val = image_transform(256,is_train=False, mean=[0.48145466,0.4578275,0.40821073], std=[0.26862954,0.26130258,0.27577711])
del mobile_model

model_name:swin_tiny_patch4_window7_224


In [9]:
preprocess_val = transforms.Compose([
    transforms.Resize(size=256, interpolation=transforms.InterpolationMode.BICUBIC),
    transforms.CenterCrop(size=(256, 256)),
    transforms.Lambda(lambda image: image.convert('RGB')),  # Assuming _convert_to_rgb is this
    transforms.ToTensor()
])

In [10]:
image = preprocess_val(Image.open("/home/alex/data/LightClip/ml-mobileclip/docs/fig_accuracy_latency.png").convert('RGB')).unsqueeze(0).half().to(device)
tokenizer = mobileclip.get_tokenizer('mobileclip_s0')

text = tokenizer(["a diagram", "a dog", "a cat"]).to(device)

with torch.no_grad(), torch.cuda.amp.autocast():
    image_features = model.encode_image(image)
    text_features = model.encode_text(text)
    image_features /= image_features.norm(dim=-1, keepdim=True)
    text_features /= text_features.norm(dim=-1, keepdim=True)

    text_probs = (100.0 * image_features @ text_features.T).softmax(dim=-1)

print("Label probs:", text_probs)

Label probs: tensor([[0.4369, 0.0360, 0.5271]], device='cuda:0')


In [11]:
checkpoint = torch.load('/home/alex/data/LightClip/CLIP-KD/logs/2024_05_13-20_09_27-t_model_ViT-B-32-256-s_model_timm-swin_tiny_patch4_window7_224-lr_0.001-b_256-tag_distill-new/checkpoints/epoch_32.pt', map_location=device)

In [12]:
checkpoint['state_dict'].keys()

odict_keys(['module.logit_scale', 'module.visual.model.patch_embed.0.reparam_conv.weight', 'module.visual.model.patch_embed.0.reparam_conv.bias', 'module.visual.model.patch_embed.1.reparam_conv.weight', 'module.visual.model.patch_embed.1.reparam_conv.bias', 'module.visual.model.patch_embed.2.reparam_conv.weight', 'module.visual.model.patch_embed.2.reparam_conv.bias', 'module.visual.model.network.0.0.layer_scale', 'module.visual.model.network.0.0.token_mixer.reparam_conv.weight', 'module.visual.model.network.0.0.token_mixer.reparam_conv.bias', 'module.visual.model.network.0.0.convffn.conv.conv.weight', 'module.visual.model.network.0.0.convffn.conv.bn.weight', 'module.visual.model.network.0.0.convffn.conv.bn.bias', 'module.visual.model.network.0.0.convffn.conv.bn.running_mean', 'module.visual.model.network.0.0.convffn.conv.bn.running_var', 'module.visual.model.network.0.0.convffn.conv.bn.num_batches_tracked', 'module.visual.model.network.0.0.convffn.fc1.weight', 'module.visual.model.netw

In [13]:
new_state_dict = {}
for k, v in checkpoint['state_dict'].items():
    if k.startswith("module."):
        new_state_dict[k[7:]] = v  # 去掉 "module." 前缀
    else:
        new_state_dict[k] = v

In [14]:
model.load_state_dict(new_state_dict)

<All keys matched successfully>

In [15]:
image = preprocess_val(Image.open("/home/alex/data/LightClip/ml-mobileclip/docs/test_cap.png").convert('RGB')).unsqueeze(0).half().to(device)
tokenizer = mobileclip.get_tokenizer('mobileclip_s0')

text = tokenizer(["a photo of diagram", "a photo of dog", "a photo of cat", "a photo of girl"]).to(device)

with torch.no_grad(), torch.cuda.amp.autocast():
    image_features = model.encode_image(image)
    text_features = model.encode_text(text)
    image_features /= image_features.norm(dim=-1, keepdim=True)
    text_features /= text_features.norm(dim=-1, keepdim=True)

    text_probs = (100.0 * image_features @ text_features.T).softmax(dim=-1)

print("Label probs:", text_probs)
for prob in text_probs[0]:
    print(prob.item())

Label probs: tensor([[2.0122e-05, 4.9518e-04, 6.9832e-04, 9.9879e-01]], device='cuda:0')
2.012164804909844e-05
0.0004951799055561423
0.000698315619956702
0.9987863898277283


In [16]:
preprocess_val = image_transform(256,is_train=False, mean=[0.48145466,0.4578275,0.40821073], std=[0.26862954,0.26130258,0.27577711])
image = preprocess_val(Image.open("/home/alex/data/LightClip/ml-mobileclip/docs/test_cap.png").convert('RGB')).unsqueeze(0).half().to(device)
tokenizer = mobileclip.get_tokenizer('mobileclip_s0')

text = tokenizer(["a photo of diagram", "a photo of dog", "a photo of cat", "a photo of girl"]).to(device)

with torch.no_grad(), torch.cuda.amp.autocast():
    image_features = model.encode_image(image)
    text_features = model.encode_text(text)
    image_features /= image_features.norm(dim=-1, keepdim=True)
    text_features /= text_features.norm(dim=-1, keepdim=True)

    text_probs = (100.0 * image_features @ text_features.T).softmax(dim=-1)

print("Label probs:", text_probs)
for prob in text_probs[0]:
    print(prob.item())


Label probs: tensor([[2.3136e-05, 9.6852e-04, 1.2436e-03, 9.9776e-01]], device='cuda:0')
2.3136184609029442e-05
0.000968523439951241
0.0012436088873073459
0.9977647066116333


In [21]:
image_features, text_features, logit_scale = model(image.float(),text)

In [22]:
text_probs = (100.0 * image_features @ text_features.T).softmax(dim=-1)

In [23]:
text_probs

tensor([[2.2544e-05, 9.6849e-04, 1.2010e-03, 9.9781e-01]], device='cuda:0',
       grad_fn=<SoftmaxBackward0>)

In [25]:
text_probs = (logit_scale.mean() * image_features @ text_features.t()).t()

In [26]:
text_probs

tensor([[10.5194],
        [12.3643],
        [12.4699],
        [15.7681]], device='cuda:0', grad_fn=<TBackward0>)

In [27]:
logit_scale.mean()

tensor(49.0637, device='cuda:0', grad_fn=<MeanBackward0>)

In [28]:
image_features @ text_features.t()

tensor([[0.2144, 0.2520, 0.2542, 0.3214]], device='cuda:0',
       grad_fn=<MmBackward0>)

In [29]:
image_features @ text_features.T

tensor([[0.2144, 0.2520, 0.2542, 0.3214]], device='cuda:0',
       grad_fn=<MmBackward0>)

In [32]:
(image_features @ text_features.T).softmax(dim=-1)

tensor([[0.2386, 0.2477, 0.2482, 0.2655]], device='cuda:0',
       grad_fn=<SoftmaxBackward0>)

In [31]:
(100.0 * image_features @ text_features.T).softmax(dim=-1)

tensor([[2.2544e-05, 9.6849e-04, 1.2010e-03, 9.9781e-01]], device='cuda:0',
       grad_fn=<SoftmaxBackward0>)